# Pubmed Example

## Data

In [11]:
metadata ={
    "description": "PUBMED dataset.",
    "data": {
        "Node": {
            "NodeFeature": {
                "description": "Node features of Pubmed dataset.",
                "type": "float",
                "format": "SparseTensor",
                "file": "pubmed.npz",
                "key": "node_feats"
            },
            "NodeLabel": {
                "description": "Node labels of Pubmed dataset, int ranged from 1 to 3.",
                "type": "int",
                "format": "Tensor",
                "file": "pubmed.npz",
                "key": "node_class",
                "num_classes": 3
            }
        },
        "Edge": {
            "_Edge": {
                "file": "pubmed.npz",
                "key": "edge"
            }
        },
        "Graph": {
            "_NodeList": {
                "file": "pubmed.npz",
                "key": "node_list"
            },
            "_EdgeList": {
                "file": "pubmed.npz",
                "key": "edge_list"
            }
        }
    },
    "citation": "@inproceedings{yang2016revisiting,\ntitle={Revisiting semi-supervised learning with graph embeddings},\nauthor={Yang, Zhilin and Cohen, William and Salakhudinov, Ruslan},\nbooktitle={International conference on machine learning},\npages={40--48},\nyear={2016},\norganization={PMLR}\n}"
 }
 

In [12]:
import json
import numpy as np
import torch
import scipy.sparse as sparse
from dgl.data import PubmedGraphDataset
dataset = PubmedGraphDataset()
graph = dataset[0]

  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


We can save the data file in only one file `pubmed.npz`.

In [13]:
# pubmed_feat
node_feats = sparse.csr_matrix(graph.ndata["feat"].numpy())
# pubmed_class
node_class = graph.ndata["label"].numpy()  # (19717,)
# pubmed_edge
edge = torch.stack(graph.edges()).numpy().T
# pubmed only has 1 single graph, there are some isolated nodes in the graph. 
# These isolated nodes are added as zero-vecs into the right position
node_list = np.ones((1, graph.num_nodes()))  # (1, 19717)
edge_list = np.ones((1, graph.num_edges()))  # (1, 88651)

data = {
    "node_feats": node_feats,
    "node_class": node_class,
    "edge": edge,
    "node_list": node_list,
    "edge_list": edge_list
}

In [14]:
np.savez_compressed("pubmed.npz", **data)

In [15]:
pubmed = np.load("pubmed.npz", allow_pickle=True)
pubmed.files

['node_feats', 'node_class', 'edge', 'node_list', 'edge_list']

In [16]:
!du pubmed.npz -h

4.7M	pubmed.npz


In [17]:
json.dump(metadata, open("metadata.json", "w"), indent=4)

## Task

In [18]:
task = {
    "description": "Node classification on PUBMED dataset. Planetoid split.",
    "type": "NodeClassification",
    "feature": [{
        "object": "Node",
        "attribute": "NodeFeature"
    }],
    "target": {
        "object": "Node",
        "attribute": "NodeLabel",
        "num_classes": 6
    },
    "train_set": {
        "file": "pubmed_task.npz",
        "key": "train"
    },
    "val_set": {
        "file": "pubmed_task.npz",
        "key": "val"
    },
    "test_set": {
        "file": "pubmed_task.npz",
        "key": "test"
    }
}


In [19]:
train_set = graph.ndata["train_mask"].nonzero().squeeze().numpy()
val_set = graph.ndata["val_mask"].nonzero().squeeze().numpy()
test_set = graph.ndata["test_mask"].nonzero().squeeze().numpy()

In [20]:
task_data = {
    "train": train_set,
    "val": val_set,
    "test": test_set
}
np.savez_compressed("pubmed_task.npz", **task_data)

with open("./task.json", "w") as fp:
    json.dump(task, fp, indent=4)